## Improving Water Consumption Management in Barcelona through Data Quality Enhancement and Prediction Models
#### **TFG 2023-2024**
#### **Author: Edith Ruiz Macià**

### Analysis of dataset Dataset1_v2.csv

### **About the Notebook**

The aim of this notebook (.ipynb) is to analyze and modify the dataset dataset1_v2.csv provided by Aigües de Barcelona in order to improve the quality of the data and learn its insights through its visualization.

The code is structured in different levels.

1. Given the database, we start with an Exploratory Data Analysis that will provide us a general knowledge of how the raw data looks like.
2. Then, an intensive data cleaning will be applied which will transform the data and handle null, negative, and outlier values.
3. We will enhance the data by adding columns that will add up to the understanding of the data and also, by combining outside data that might be relevant to ours.
4. In order to have a complete dataset we will make a projection of absent or incorrect values previously detected in section 2. To do so, we will test several algorithms and see with which one we obtain better results.
5. Anomalies that were previously discarded and substituted by projected values with our predictive model must also be taken into account and analyzed. That is why we will classify and visualize them.

Finally, the updated resulting database can be downloaded in order to be used in other programs or other purposes of the user.

#### ***Execution time***
bla bla bla

#### ***How to execute the code***
This is a Python Notebook, so the code should be run either cell by cell or by running all the notebook at once. At the beginning, review and make sure the paths to all datasets are correct for your folders' distribution in order for everything to run smoothly.

#### ***Libraries and packages used in the project***
Here is a brief explanation of the imported libraries:


1. **math:** It is a Python standard library that provides basic math functions

2. **datetime:** It is also part of a standard library and is used to work with dates and times

3. **collections:** Another standard library that provides specialized data types like Counter which is used to count elements in a list

3. **numpy:** Fundamental library for numerical computation. Specifically, linalg is a linear algebra module within NumPy

3. **time:** Standard library for measuring time and working with dates and times

4. **re:** Regular expression library used to search and manipulate text strings based on a given pattern

5. **json:** Standard library for working with JSON (JavaScript Object Notation) format

6. **pandas:** Very popular library for data manipulation and analysis. It is used to work with data structures such as DataFrames

7. **matplotlib.pyplot:** Used to create graphs and visualizations

8. **sklearn.model_selection:** Provides tools to split data sets into training and test subsets for a model

9. **sklearn.linear_model**: Contains implementations of linear regression models

10. **sklearn.metrics:** Contains metrics for evaluating machine learning models

11. **sklearn.preprocessing:** Provides functions to preprocess data before fitting a model

12. **sklearn.compose:** Contains utilities for building and combining transformers

13. **sklearn.feature_extraction.text:** Tools for extracting features from text data

14. **sklearn.cluster:** Contains clustering algorithms such as KMeans

15. **sklearn.metrics.pairwise:** Metrics to calculate similarity between samples

16. **seaborn:** Data visualization library based on Matplotlib. It provides a high-level interface for drawing attractive statistical graphs

17. **xgboost:** Library to implement Gradient Boosting models

18. **sklearn.ensemble:** Contains implementations of ensemble models, such as Random Forest

In [ ]:
# In case you are working in google colab you can mount your google drive account here
# from google.colab import drive
# drive.mount('/content/drive')

## 0. Importing datasets and libraries

In [ ]:
!pip install shap

In [ ]:
# Import the necessary libraries
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
import shap

In [ ]:
# Change your path of the data if necessary
dataset1 = 'C:/Users/edith/Desktop/TFG/Datasets/Aigües de Barcelona/Sets-de-dades/dataset1_v2.csv'
dataset1 = pd.read_csv(dataset1)
dataset1.head()

## 1. Exploratory Data Analysis

As a starting point, we need to explore the data. We analyze the summary of descriptive statistics and plots in order to detect the corresponding anomalies and subsequently process them correctly.

In [ ]:
dataset1.info()

In [ ]:
dataset1.describe()

In [ ]:
# Accumulated Consumption over Time
dataset1['Data/Fecha/Date'] = pd.to_datetime(dataset1['Data/Fecha/Date'])
plt.figure(figsize=(10, 6))
plt.scatter(dataset1['Data/Fecha/Date'], dataset1['Consum acumulat (L/dia)/Consumo acumulado(L/día)/Accumulated Consumption (L/day)'], color='blue', marker='o', s=50, alpha=0.7, label='Data Points')
plt.xlabel('Date', fontsize=12)
plt.ylabel('Accumulated Consumption (L/day)', fontsize=12)
plt.title('Time Series Plot of Accumulated Consumption', fontsize=15)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# Aggregate total consumption for each municipality
total_consumption_per_municipality = dataset1.groupby('Municipi/Municipio/Municipality')['Consum acumulat (L/dia)/Consumo acumulado(L/día)/Accumulated Consumption (L/day)'].sum()
total_consumption_per_municipality.plot(kind='bar', figsize=(10,6))
plt.xlabel('Municipality', fontsize=12)
plt.ylabel('Total Accumulated Consumption (L/day)', fontsize=12)
plt.title('Total Accumulated Consumption per Municipality', fontsize=15)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
print("Poscodes of Barcelona:")
print(dataset1[dataset1['Municipi/Municipio/Municipality'] == 'BARCELONA']["Codi postal/Código postal/Postcode"].unique())

In [ ]:
# Aggregate total consumption for each type of use
total_consumption_per_use = dataset1.groupby('Ús/Uso/Use')['Consum acumulat (L/dia)/Consumo acumulado(L/día)/Accumulated Consumption (L/day)'].sum()
plt.figure(figsize=(10, 6))
total_consumption_per_use.plot(kind='pie', autopct='%1.1f%%', startangle=140, colors=['lightskyblue', 'blue', 'cornflowerblue'])
plt.title('Total Accumulated Consumption per Type of Use', fontsize=15)
plt.ylabel(None)
plt.tight_layout()
plt.axis('equal')
plt.show()

**Insights:**
- Negative consumption values are considered as incorrect values for the Cumulative Consumption (L/day) column. Consumption has to be positive.
- The number of meters ('Number of meters') varies greatly, with some locations having just one meter and others up to 1254 meters.
- In the scatter plot we can clearly see outliers that will be removed later on.
- Most of the consumption is accumulated in the Municipality of Barcelona and it's of Domestic use.

## 2. Data Preprocessing

##### **2.1. Data Transformation**

In [ ]:
# Rename dataset1_v2 columns
new_column_names = {
    'Secció Censal/Sección censal/Census section': 'Census section',
    'Districte/Distrito/District': 'District',
    'Codi postal/Código postal/Postcode': 'Postcode',
    'Municipi/Municipio/Municipality': 'Municipality',
    'Data/Fecha/Date': 'Date',
    'Ús/Uso/Use': 'Use',
    'Nombre de comptadors/Número de contadores/Number of meters': 'Number of meters',
    'Consum acumulat (L/dia)/Consumo acumulado(L/día)/Accumulated Consumption (L/day)': 'Accumulated Consumption (L/day)'
}

dataset1.rename(columns=new_column_names, inplace=True)

In [ ]:
# Randomly sample 700000 data points
sampled_indices = np.random.choice(len(dataset1), size=700000, replace=False)
sampled_data = dataset1.iloc[sampled_indices]

# Scatter Plot for Accumulated Consumption vs Number of Meters without negatives
plt.figure(figsize=(10, 6))
plt.scatter(sampled_data['Number of meters'], sampled_data['Accumulated Consumption (L/day)'], color='blue', marker='o', s=50, alpha=0.7, label='Data Points')
plt.title('Accumulated Consumption vs Number of Meters without negatives', fontsize=15)
plt.xlabel('Number of Meters', fontsize=12)
plt.ylabel('Accumulated Consumption (L/day)', fontsize=12)
plt.ylim(0, 100000)

# Calculate the trend line
x = sampled_data['Number of meters']
y = sampled_data['Accumulated Consumption (L/day)']
m, b = np.polyfit(x, y, 1)
plt.plot(x, m*x + b, color='red')

plt.show()

We cannot assure that consumption is not normalized, but according to the plots above it does not look like it therefore, they have to be normalized. We will divide the Accumulated Consumption by the Number of Meters in order to obtain the Accumulated Consumption for a single Meter.

In [ ]:
# Normalization of Accumulated Consumption (L/day) by Number of meters
dataset1['Normalized Accumulated Consumption (L/day)'] = round(dataset1['Accumulated Consumption (L/day)'] / dataset1['Number of meters'], 3)
dataset1.head()

In [ ]:
# Distribution of normalized vs non normalized
plt.figure(figsize=(10, 5))
plt.scatter(dataset1["Date"], dataset1["Accumulated Consumption (L/day)"], color='blue',  marker='o', s=20, alpha=0.7, label='Accumulated Consumption')
plt.scatter(dataset1["Date"], dataset1["Normalized Accumulated Consumption (L/day)"], color='lightskyblue',  marker='o', s=20, alpha=0.7, label='Normalized Accumulated Consumption')
plt.title("Distribution of Accumulated Consumption vs Normalized Accumulated Consumption over Time", fontsize=12)
plt.xlabel("Date", fontsize=10)
plt.ylabel("Accumulated Consumption (L/day)", fontsize=10)
plt.xticks(rotation=45, ha='right', fontsize=8)
plt.yticks(fontsize=8)
plt.grid(True)
plt.legend(fontsize=7)
tick_frequency = 100
plt.gca().xaxis.set_major_locator(mdates.DayLocator(interval=tick_frequency))
plt.show()

In [ ]:
plt.figure(figsize=(10, 5))
plt.scatter(dataset1["Date"], dataset1["Normalized Accumulated Consumption (L/day)"], color='blue',  marker='o', s=20, alpha=0.7, label='Normalized Accumulated Consumption')
plt.title("Distribution of Accumulated Consumption vs Normalized Accumulated Consumption over Time", fontsize=12)
plt.xlabel("Date", fontsize=10)
plt.ylabel("Accumulated Consumption (L/day)", fontsize=10)
plt.xticks(rotation=45, ha='right', fontsize=8)
plt.yticks(fontsize=8)
plt.grid(True)
plt.legend(fontsize=7)
tick_frequency = 100
plt.gca().xaxis.set_major_locator(mdates.DayLocator(interval=tick_frequency))
plt.show()

In [ ]:
dataset1_box = dataset1.dropna(subset = ['District'])
dataset1_box = dataset1_box.dropna(subset = ['Census section'])

use_cases = dataset1_box['Use'].unique()
num_plots = len(use_cases)
num_columns = 3
num_rows = int(np.ceil(num_plots / num_columns))

fig, axes = plt.subplots(num_rows, num_columns, figsize=(15, num_rows * 7))
axes = axes.flatten()

for i, use_case in enumerate(use_cases):
    iteration_dataset = dataset1_box[dataset1_box['Use'] == use_case].copy()
    columns_to_plot = ['Normalized Accumulated Consumption (L/day)']
    axes[i].boxplot([iteration_dataset[col] for col in columns_to_plot], labels=columns_to_plot)
    axes[i].set_title(f'Boxplot of {use_case}', fontsize=12)
    axes[i].set_xlabel('Columns', fontsize=10)
    axes[i].set_ylabel('Values', fontsize=10)
    axes[i].tick_params(axis='x', labelsize=8)
    axes[i].tick_params(axis='y', labelsize=8)

# Hide any unused subplots if the number of use cases is not a multiple of num_columns
for j in range(i + 1, num_rows * num_columns):
    fig.delaxes(axes[j])

plt.tight_layout()
plt.show()

In [ ]:
dataset1.info()

##### **2.2. Null Values**

In [ ]:
# Null values per column
columns_dataset1 = dataset1.columns
for column in columns_dataset1:
  print("Column:", column, "- Null values: ", dataset1[column].isnull().sum())

##### **2.3. Wrong Values**

Now we want to identify the erroneous values in our data. We consider as wrong values the negative consumptions. We will store those values for later analysis.

In [ ]:
# We identify negative values in Normalized Accumulated Consumption (L/day)
num_negative_consum_rows = len(dataset1[dataset1['Normalized Accumulated Consumption (L/day)'] < 0])
print("Negative in 'Normalized Accumulated Consumption (L/day)' in the dataset:", num_negative_consum_rows)

In [ ]:
# Copy the negative values into the anomalies dataset for later analysis
anomalies = dataset1[dataset1['Normalized Accumulated Consumption (L/day)'] < 0].copy()
anomalies.head()

In [ ]:
# Replace negative values in Normalized Accumulated Consumption (L/day) by null
dataset1_1 = dataset1.copy()
dataset1_1.loc[dataset1_1['Normalized Accumulated Consumption (L/day)'] < 0, 'Normalized Accumulated Consumption (L/day)'] = np.nan

# Let's put null also the values of the Accumulated Consumption (L/day)
dataset1_1.loc[dataset1_1['Accumulated Consumption (L/day)'] < 0, 'Accumulated Consumption (L/day)'] = np.nan

# Number of negative values in Normalized Accumulated Consumption (L/day) after removing negative values
num_negative_consum_rows2 = len(dataset1_1[dataset1_1['Normalized Accumulated Consumption (L/day)'] < 0])
print("Negative in 'Normalized Accumulated Consumption (L/day)' in the dataset:", num_negative_consum_rows2)

# Number of null values in Normalized Accumulated Consumption (L/day) after removing negative values
print("Number of Nulls in the dataset without negative values:", dataset1_1['Normalized Accumulated Consumption (L/day)'].isnull().sum())

##### **2.4. Outliers**

In [ ]:
# Treatment of outliers in Normalized Accumulated Consumption (L/day) with the IQR method

# We separate according to USE: Industrial, Domestic or Commercial
domestic_df = dataset1_1[dataset1_1['Use'].str.contains('Domestic', case=False, na=False)].copy()
industrial_df = dataset1_1[dataset1_1['Use'].str.contains('Industrial', case=False, na=False)].copy()
comercial_df = dataset1_1[dataset1_1['Use'].str.contains('Commercial', case=False, na=False)].copy()

def outliers_iqr(dataframe):
    global anomalies

    # We compute the IQR for the 'Normalized Accumulated Consumption (L/day)' column
    consum_col = 'Normalized Accumulated Consumption (L/day)'
    Q1 = dataframe[consum_col].quantile(0.25)
    Q3 = dataframe[consum_col].quantile(0.75)
    IQR = Q3 - Q1

    # We identify the outliers
    outlier_filter = ((dataframe[consum_col] < (Q1 - 1.5 * IQR)) | (dataframe[consum_col] > (Q3 + 1.5 * IQR)))

    # Copy the outlier value into the anomalies dataset for later analysis
    outliers = dataframe[outlier_filter].copy()
    anomalies = pd.concat([anomalies, outliers], ignore_index=True)

    # And replace them with null
    dataframe.loc[outlier_filter, consum_col] = np.nan

    return dataframe

# We apply the function for each dataframe corresponding to each "Use"
domestic_df = outliers_iqr(domestic_df)
industrial_df = outliers_iqr(industrial_df)
comercial_df = outliers_iqr(comercial_df)

dataset1_filtered = pd.concat([domestic_df, industrial_df, comercial_df], ignore_index=True)

In [ ]:
# Let's put to null also the values that correspond to those outliers in 'Accumulated Consumption (L/day)'
dataset1_filtered.loc[dataset1_filtered['Normalized Accumulated Consumption (L/day)'].isnull(), 'Accumulated Consumption (L/day)'] = np.nan

In [ ]:
anomalies.info()

In [ ]:
# Number of null values in Normalized Accumulated Consumption (L/day) prior to removing outliers
print("Number of Nulls in the dataset with outliers but no negatives:", dataset1_1['Normalized Accumulated Consumption (L/day)'].isnull().sum())        #dataset with outliers

# Number of null values in Normalized Accumulated Consumption (L/day) after removing outliers
print("Number of Nulls in the dataset without outliers nor negatives:", dataset1_filtered['Normalized Accumulated Consumption (L/day)'].isnull().sum())   #dataset without outliers

In [ ]:
dataset1_box = dataset1_filtered.dropna(subset = ['District'])
dataset1_box = dataset1_box.dropna(subset = ['Census section'])
dataset1_box = dataset1_box.dropna(subset = ['Accumulated Consumption (L/day)'])
dataset1_box = dataset1_box.dropna(subset = ['Normalized Accumulated Consumption (L/day)'])

use_cases = dataset1_box['Use'].unique()
num_plots = len(use_cases)
num_columns = 3
num_rows = int(np.ceil(num_plots / num_columns))

fig, axes = plt.subplots(num_rows, num_columns, figsize=(15, num_rows * 7))
axes = axes.flatten()

for i, use_case in enumerate(use_cases):
    iteration_dataset = dataset1_box[dataset1_box['Use'] == use_case].copy()
    columns_to_plot = ['Normalized Accumulated Consumption (L/day)']
    axes[i].boxplot([iteration_dataset[col] for col in columns_to_plot], labels=columns_to_plot)
    axes[i].set_title(f'Boxplot of {use_case}', fontsize=12)
    axes[i].set_xlabel('Columns', fontsize=10)
    axes[i].set_ylabel('Values', fontsize=10)
    axes[i].tick_params(axis='x', labelsize=8)
    axes[i].tick_params(axis='y', labelsize=8)

# Hide any unused subplots if the number of use cases is not a multiple of num_columns
for j in range(i + 1, num_rows * num_columns):
    fig.delaxes(axes[j])

plt.tight_layout()
plt.show()

##### **2.5. Storage of Anomalies**

If all anomalies in the data have been cleaned correctly we should have the sum of the number of outlier values and negatives remaining in our anomalies stored.

Negative values removed = 6222

Outliers removed = 334898 - 6222 = 328676

Total number of anomalies = 334898

In [ ]:
# Save current clean dataset for classification of anomalies
clean_data = dataset1_filtered.copy()

anomalies.head()

## 3. Data Enhancement

In [ ]:
# Convert "Date" column to datetime
dataset1_filtered['Date'] = pd.to_datetime(dataset1_filtered['Date'])
dataset1_filtered.info()

In [ ]:
# We create a new "Season" column based on the "Date" column
def map_to_season(month):
    if 3 <= month <= 5:
        return 'Spring'
    elif 6 <= month <= 8:
        return 'Summer'
    elif 9 <= month <= 11:
        return 'Autumn'
    else:
        return 'Winter'

dataset1_filtered['Season'] = dataset1_filtered['Date'].dt.month.map(map_to_season)

# We create a new column "Day_of_Week" based on the column "Date"
dataset1_filtered['Day of Week'] = dataset1_filtered['Date'].dt.day_name()

# We create a new column "Month" based on the column "Date"
dataset1_filtered['Month'] = dataset1_filtered['Date'].dt.month

# We create a new column "Year" based on the column "Date"
dataset1_filtered['Year'] = dataset1_filtered['Date'].dt.year

dataset1_filtered.head()

As we can see now all data in this dataset ranges from years 2019 to 2022. This is important as now we are going to add new information from other datasets and we are going to need the dates to match.

In [ ]:
dataset1_filtered['Year'].unique()

In [ ]:
temperature = 'C:/Users/edith/Desktop/TFG\Datasets/Dades meteorològiques de la XEMA/Temperature_Dades_Meteorologiques_XEMA.csv'
temperature = pd.read_csv(temperature)
temperature['DATA_LECTURA'] = pd.to_datetime(temperature['DATA_LECTURA'])
temperature['DATA_LECTURA'] = temperature['DATA_LECTURA'].dt.date
temperature['DATA_LECTURA'] = pd.to_datetime(temperature['DATA_LECTURA'])
temperature = temperature.groupby(['CODI_ESTACIO', 'DATA_LECTURA'])['VALOR_LECTURA'].mean().reset_index()
temperature.head(5)

In [ ]:
rel_humidity = 'C:/Users/edith/Desktop/TFG\Datasets/Dades meteorològiques de la XEMA/Relative_Humidity_Dades_Meteorologiques_XEMA.csv'
rel_humidity = pd.read_csv(rel_humidity)
rel_humidity['DATA_LECTURA'] = pd.to_datetime(rel_humidity['DATA_LECTURA'])
rel_humidity['DATA_LECTURA'] = rel_humidity['DATA_LECTURA'].dt.date
rel_humidity = rel_humidity.groupby(['CODI_ESTACIO', 'DATA_LECTURA'])['VALOR_LECTURA'].mean().reset_index()
rel_humidity.head(5)

In [ ]:
atm_pressure = 'C:/Users/edith/Desktop/TFG\Datasets/Dades meteorològiques de la XEMA/Atmospheric_Pressure_Dades_Meteorologiques_XEMA.csv'
atm_pressure = pd.read_csv(atm_pressure)
atm_pressure['DATA_LECTURA'] = pd.to_datetime(atm_pressure['DATA_LECTURA'])
atm_pressure['DATA_LECTURA'] = atm_pressure['DATA_LECTURA'].dt.date
atm_pressure = atm_pressure.groupby(['CODI_ESTACIO', 'DATA_LECTURA'])['VALOR_LECTURA'].mean().reset_index()
atm_pressure.head(5)

In [ ]:
precipitation = 'C:/Users/edith/Desktop/TFG\Datasets/Dades meteorològiques de la XEMA/Precipitation_Dades_Meteorologiques_XEMA.csv'
precipitation = pd.read_csv(precipitation)
precipitation['DATA_LECTURA'] = pd.to_datetime(precipitation['DATA_LECTURA'])
precipitation['DATA_LECTURA'] = precipitation['DATA_LECTURA'].dt.date
precipitation = precipitation.groupby(['CODI_ESTACIO', 'DATA_LECTURA'])['VALOR_LECTURA'].mean().reset_index()
precipitation.head(5)

In [ ]:
solar_rad = 'C:/Users/edith/Desktop/TFG\Datasets/Dades meteorològiques de la XEMA/Solar_Radation_global_Dades_Meteorologiques_XEMA.csv'
solar_rad = pd.read_csv(solar_rad)
solar_rad['DATA_LECTURA'] = pd.to_datetime(solar_rad['DATA_LECTURA'])
solar_rad['DATA_LECTURA'] = solar_rad['DATA_LECTURA'].dt.date
solar_rad = solar_rad.groupby(['CODI_ESTACIO', 'DATA_LECTURA'])['VALOR_LECTURA'].mean().reset_index()
solar_rad.head(5)

In [ ]:
postalcode_to_station = {
    8001: 'X4', 8002: 'X4', 8003: 'X2', 8004: 'X4', 8005: 'X2', 8006: 'D5', 8007: 'X4', 8008: 'X4', 8009: 'X4', 8010: 'X4',
    8011: 'X4', 8012: 'D5', 8013: 'X2', 8014: 'XL', 8015: 'X4', 8016: 'D5', 8017: 'X8', 8018: 'X2', 8019: 'X2', 8020: 'X2',
    8021: 'X8', 8022: 'D5', 8023: 'D5', 8024: 'D5', 8025: 'D5', 8026: 'X2', 8027: 'D5', 8028: 'XL', 8029: 'X8', 8030: 'D5',
    8031: 'D5', 8032: 'D5', 8033: 'D5', 8034: 'X8', 8035: 'D5', 8036: 'X4', 8037: 'X4', 8038: 'X4', 8039: 'X4', 8040: 'XL',
    8041: 'D5', 8042: 'D5'
}


In [ ]:
def mapping_geotemporal(dataset, geotemporal, attribute):

    # if dataset['Date'].dtype == 'datetime64[ns]':
    #     if any(dataset['Date'].dt.time != pd.Timestamp(0).time()):
    #         dataset["Date"] = dataset["Date"].dt.date

    for index, row in dataset.iterrows():
        municipality = row["Municipality"]
        date = row["Date"]
        postcode = row["Postcode"]

        if municipality == "GAVA":
            filtered_row = geotemporal[(geotemporal["CODI_ESTACIO"] == "UG") & (geotemporal["DATA_LECTURA"] == date)]
            if not filtered_row.empty:
                dataset.at[index, attribute] = filtered_row.iloc[0]["VALOR_LECTURA"]
        
        elif municipality == "VILADECANS":
            filtered_row = geotemporal[(geotemporal["CODI_ESTACIO"] == "UG") & (geotemporal["DATA_LECTURA"] == date)]
            if not filtered_row.empty:
                dataset.at[index, attribute] = filtered_row.iloc[0]["VALOR_LECTURA"]
        
        elif municipality == "SANT ADRIA":
            filtered_row = geotemporal[(geotemporal["CODI_ESTACIO"] == "WU") & (geotemporal["DATA_LECTURA"] == date)]
            if not filtered_row.empty:
                dataset.at[index, attribute] = filtered_row.iloc[0]["VALOR_LECTURA"]
        
        elif municipality == "L'HOSPITALET LLOBR.":
            filtered_row = geotemporal[(geotemporal["CODI_ESTACIO"] == "XL") & (geotemporal["DATA_LECTURA"] == date)]
            if not filtered_row.empty:
                dataset.at[index, attribute] = filtered_row.iloc[0]["VALOR_LECTURA"]
        
        elif municipality == "SANT FELIU LL.":
            filtered_row = geotemporal[(geotemporal["CODI_ESTACIO"] == "X8") & (geotemporal["DATA_LECTURA"] == date)]
            if not filtered_row.empty:
                dataset.at[index, attribute] = filtered_row.iloc[0]["VALOR_LECTURA"]
                        
        elif municipality == "BARCELONA":
            station_code = postalcode_to_station.get(postcode)
            filtered_row = geotemporal[(geotemporal["CODI_ESTACIO"] == station_code) & (geotemporal["DATA_LECTURA"] == date)]
            if not filtered_row.empty:
                dataset.at[index, attribute] = filtered_row.iloc[0]["VALOR_LECTURA"]


mapping_geotemporal(dataset1_filtered, temperature, "Temperature")
dataset1_filtered

---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

Lets add information to the dataset about the precipitations in Barcelona for the dates of each entrie as well as information about the temperatures. Both precipitation and temperatures should have a strong correlation with the consumed water.

In [ ]:
# Change your path of the data if necessary
dataset_rain = 'C:/Users/edith/Desktop/TFG/Datasets/OpenDataBCN/precipitacionsbcndesde1786_2023_long.csv'
dataset_rain = pd.read_csv(dataset_rain)
dataset_rain.head()

In [ ]:
# Filter the dataset to keep only the desired years
years_to_keep = [2019, 2020, 2021, 2022]
dataset_rain = dataset_rain[dataset_rain['Any'].isin(years_to_keep)]

# Display the first few rows of the filtered dataset
dataset_rain.head()

In [ ]:
def map_to_precipitation(dataset_filtered, dataset_rain):

    # Merge datasets on 'Year' and 'Month'
    merged_data = pd.merge(dataset_filtered, dataset_rain, left_on=['Year', 'Month'], right_on=['Any', 'Mes'], how='left')

    # Drop unnecessary columns from the merged dataset
    merged_data.drop(['Any', 'Mes', 'Desc_Mes'], axis=1, inplace=True)

    return merged_data

dataset1_filtered = map_to_precipitation(dataset1_filtered, dataset_rain)
dataset1_filtered.rename(columns = {'Precipitacions': 'Precipitations'}, inplace=True)
dataset1_filtered.head()

In [ ]:
# Precipitation Over Time
df_sorted = dataset1_filtered.sort_values(by='Date')
plt.figure(figsize=(10, 5))
plt.plot(df_sorted['Date'], df_sorted['Precipitations'], color='blue', linewidth=2)
plt.xlabel('Date', fontsize=10)
plt.ylabel('Precipitations (mm)', fontsize=10)
plt.title('Precipitations Over Time', fontsize=12)
plt.xticks(fontsize=8)
plt.yticks(fontsize=8)
plt.show()

In [ ]:
# Change your path of the data if necessary
dataset_temp = 'C:/Users/edith/Desktop/TFG/Datasets/OpenDataBCN/temperaturesbcndesde1780_2023_long.csv'
dataset_temp = pd.read_csv(dataset_temp)
dataset_temp.head()

In [ ]:
# Filter the dataset to keep only the desired years
years_to_keep = [2019, 2020, 2021, 2022]
dataset_temp = dataset_temp[dataset_temp['Any'].isin(years_to_keep)]

# Display the first few rows of the filtered dataset
dataset_temp.head()

In [ ]:
def map_to_temperature(dataset_filtered, dataset_temp):

    # Merge datasets on 'Year' and 'Month'
    merged_data = pd.merge(dataset_filtered, dataset_temp, left_on=['Year', 'Month'], right_on=['Any', 'Mes'], how='left')

    # Drop unnecessary columns from the merged dataset
    merged_data.drop(['Any', 'Mes', 'Desc_Mes'], axis=1, inplace=True)

    return merged_data

dataset1_filtered = map_to_temperature(dataset1_filtered, dataset_temp)
dataset1_filtered.rename(columns = {'Temperatura': 'Temperature'}, inplace=True)
dataset1_filtered.head()

In [ ]:
# Temperature Over Time
df_sorted = dataset1_filtered.sort_values(by='Date')
plt.figure(figsize=(10, 5))
plt.plot(df_sorted['Date'], df_sorted['Temperature'], color='blue', linewidth=2)
plt.xlabel('Date', fontsize=10)
plt.ylabel('Temperature (ºC)', fontsize=10)
plt.title('Temperature Over Time', fontsize=12)
plt.xticks(fontsize=8)
plt.yticks(fontsize=8)
plt.show()

## 4. Data Prediction

In order to do the projection of incorrect values (negative and outliers) that right now happen to be null values in the dataset, we will implement several data prediction models, compare their results and conclude which gives the best results for our data.

We will now encode the variables that aren't numerical in order to work with them and plot a correlation matrix to see which features explain best our target variable, Accumulated Consumption (L/day). We are making a prediction with Accumulated Consumption (L/day) instead of the normalized data because it was the original one and shows more promising results. At the end we just have to divide by the Number of Meters.

In [ ]:
# Convert "Date" column to object for label encoding
dataset1_filtered['Date'] = dataset1_filtered['Date'].astype(object)

In [ ]:
# Label Encoder

# Initialize LabelEncoder
label_encoder = LabelEncoder()

# Apply LabelEncoder to each column containing categorical data
for column in dataset1_filtered.columns:
    if dataset1_filtered[column].dtype == 'object':
        dataset1_filtered[column] = label_encoder.fit_transform(dataset1_filtered[column])

In [ ]:
# We remove the null values to see the correct correlation of the data
dataset1_filtered = dataset1_filtered.dropna(subset = ['District'])
dataset1_filtered = dataset1_filtered.dropna(subset = ['Census section'])

dataset1_filtered_not_null = dataset1_filtered[~dataset1_filtered['Accumulated Consumption (L/day)'].isnull()]
corr = dataset1_filtered_not_null.corr()
plt.figure(figsize=(15,6))
heatmap = sns.heatmap(corr, vmin=-1, vmax=1, annot=True, cmap='coolwarm')
heatmap.set_title('Correlation Heatmap', fontdict={'fontsize': 15})
plt.show()

In [ ]:
# Model Training

# We remove the null values in the consumption column to train the model
dataset1_filtered2 = dataset1_filtered.dropna(subset=['Accumulated Consumption (L/day)'])

# Independent variables 
features = ['Use', 'Number of meters', 'Date', 'Postcode', 'Census section']

X = dataset1_filtered2[features]

# Target variable (dependent variable)
y = dataset1_filtered2['Accumulated Consumption (L/day)']

# 80% training and 20% testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# We make sure that X and Y have the same length for both train and test
print("Lenght X_train:", len(X_train))
print("Length y_train:", len(y_train))
print("Lenght X_test:", len(X_test))
print("Length y_test:", len(y_test))

##### **4.1. Linear Regression**

In [ ]:
# Linear Regression
lr_model = LinearRegression()
lr_model.fit(X_train, y_train)
y_pred = lr_model.predict(X_test)

# Evalute the model
mse_lr = mean_squared_error(y_test, y_pred)
mae_lr = mean_absolute_error(y_test, y_pred)
r2_lr = r2_score(y_test, y_pred)

print(f"Linear Regression Mean Squared Error (MSE): {round(np.sqrt(mse_lr), 4)}")
print(f"Linear Regression Mean Absolute Error (MAE): {round(mae_lr, 4)}")
print(f"Linear Regression R-squared (R2): {round(r2_lr, 4)}")

##### **4.2. Random Forest**

In [ ]:
# Random Forest
rf_model = RandomForestRegressor(random_state=42)
rf_model.fit(X_train, y_train)
y_pred_rf = rf_model.predict(X_test)

# Evaluate the model
mse_rf = mean_squared_error(y_test, y_pred_rf)
mae_rf = mean_absolute_error(y_test, y_pred_rf)
r2_rf = r2_score(y_test, y_pred_rf)

print(f"Random Forest Mean Squared Error (MSE):", round(np.sqrt(mse_rf), 4))
print(f"Random Forest Mean Absolute Error (MAE): {round(mae_rf, 4)}")
print(f"Random Forest R-squared (R2):", round(r2_rf, 4))

##### **4.3. XGBoost**

In [ ]:
# XGBoost
xgb_model = xgb.XGBRegressor(random_state=42)
xgb_model.fit(X_train, y_train)
y_pred_xgb = xgb_model.predict(X_test)

# Evaluate the model
mse_xgb = mean_squared_error(y_test, y_pred_xgb)
mae_xgb = mean_absolute_error(y_test, y_pred_xgb)
r2_xgb = r2_score(y_test, y_pred_xgb)

print(f"XGBoost Mean Squared Error (MSE):", round(np.sqrt(mse_xgb), 4))
print(f"XGBoost Mean Absolute Error (MAE): {round(mae_xgb, 4)}")
print(f"XGBoost R-squared (R2):", round(r2_xgb, 4))

##### **4.4. K-Nearest Neighbors**

In [ ]:
# Range of k values to test
k_values = range(1, 10)
mse_values = []

# Evaluation of the model for each k value
for k in k_values:
    knn_model = KNeighborsRegressor(n_neighbors=k)
    knn_model.fit(X_train, y_train)
    y_pred_knn = knn_model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred_knn)
    mse_values.append(mse)

# Plot error rates vs k values
plt.figure(figsize=(10, 6))
plt.plot(k_values, mse_values, marker='o', linestyle='dashed', color='b')
plt.xlabel('Number of Neighbors K', fontsize=10)
plt.ylabel('Mean Squared Error (MSE)', fontsize=10)
plt.title('Error Rate vs. K Value', fontsize=12)
plt.xticks(k_values)
plt.grid(True)
plt.show()

# k value with the minimum error rate
best_k = k_values[np.argmin(mse_values)]
print(f"The best k value is: {best_k}")

In [ ]:
# K-Nearest Neighbors
knn_model = KNeighborsRegressor(n_neighbors=2)
knn_model.fit(X_train, y_train)
y_pred_knn = knn_model.predict(X_test)

# Evaluate the model
mse_knn = mean_squared_error(y_test, y_pred_knn)
mae_knn = mean_absolute_error(y_test, y_pred_knn)
r2_knn = r2_score(y_test, y_pred_knn)

print(f"K-Nearest Neighbors Mean Squared Error (MSE):", round(np.sqrt(mse_knn), 4))
print(f"K-Nearest Neighbors Mean Absolute Error (MAE): {round(mae_knn, 4)}")
print(f"K-Nearest Neighbors R-squared (R2):", round(r2_knn, 4))

##### **4.5. Support Vector Machine**

In [ ]:
# Model Training

# We remove the null values in the consumption column to train the model
dataset1_filtered2 = dataset1_filtered.dropna(subset=['Accumulated Consumption (L/day)'])

# Reduce the working dataset
rows = len(dataset1_filtered2) // 10
dataset1_filtered2 = dataset1_filtered2.iloc[:rows]

# Independent variables
features = ['Use', 'Number of meters', 'Date', 'Postcode', 'Census section']
X = dataset1_filtered2[features]

# Target variable (dependent variable)
y = dataset1_filtered2['Accumulated Consumption (L/day)']

# 80% training and 20% testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# We make sure that X and Y have the same length for both train and test
print("Lenght X_train:", len(X_train))
print("Length y_train:", len(y_train))
print("Lenght X_test:", len(X_test))
print("Length y_test:", len(y_test))

In [ ]:
# Support Vector Machine
svm_model = SVR()
svm_model.fit(X_train, y_train)
y_pred_svm = svm_model.predict(X_test)

# Evaluate the model
mse_svm = mean_squared_error(y_test, y_pred_svm)
mae_svm = mean_absolute_error(y_test, y_pred_svm)
r2_svm = r2_score(y_test, y_pred_svm)

print(f"Support Vector Machine Mean Squared Error (MSE):", round(np.sqrt(mse_svm), 4))
print(f"Support Vector Machine Mean Absolute Error (MAE): {round(mae_svm, 4)}")
print(f"Support Vector Machine R-squared (R2):", round(r2_svm, 4))

##### **4.6. Explainability**

In [ ]:
# Convert the data to numpy arrays
X_train_array = X_train.values
X_test_array = X_test.values

# We use a small subset of the training data to initialize the SHAP KernelExplainer
background = shap.kmeans(X_train_array, 10)
explainer = shap.KernelExplainer(knn_model.predict, background)

In [ ]:
# We compute SHAP values for the test set
shap_values = explainer.shap_values(X_test_array)

In [ ]:
# Convert shap_values to Explanation objects
shap_values_exp = shap.Explanation(values=shap_values, base_values=explainer.expected_value, data=X_test_array, feature_names=features)

# Visualize SHAP values
shap.initjs()

In [ ]:
shap.summary_plot(shap_values, X_test)

In [ ]:
features = ['Use', 'Number of meters', 'Date', 'Postcode', 'Census section']

# Plot layout
fig, axs = plt.subplots(2, 3, figsize=(20, 10))  # 2 rows, 3 columns
axs = axs.flatten()

# Dependence plots
for i, ft in enumerate(features):
    shap.dependence_plot(ft, shap_values, X_test, ax=axs[i], show=False)

# Hide the empty subplot (the 6th plot space)
fig.delaxes(axs[-1])

# Adjust layout
plt.tight_layout()
plt.show()

In [ ]:
# For a single prediction
shap.force_plot(explainer.expected_value, shap_values[0], X_test.iloc[0])

In [ ]:
shap.waterfall_plot(shap_values_exp[0])

In [ ]:
shap.decision_plot(shap_values_exp[0].base_values, shap_values_exp[0].values, X_test.iloc[0])

In [ ]:
features = ['Use', 'Number of meters', 'Date', 'Postcode', 'Census section']

# Plot layout
fig, axs = plt.subplots(2, 3, figsize=(20, 10))  # 2 rows, 3 columns
axs = axs.flatten()

# Scatter plots
for i, ft in enumerate(features):
    shap.plots.scatter(shap_values_exp[:, ft], ax=axs[i], show=False)

# Hide the empty subplot (the 6th plot space)
fig.delaxes(axs[-1])

# Adjust layout
plt.tight_layout()
plt.show()

##### **4.7. Conclusions on Data Prediction Algorithms**

Based on the R-squared values obtained per each model I conclude that the K-Nearest Neighbors model, with 0.9705, is the most accurate for dataset1_v2.csv data. It is considered a good model if the R-squared value is higher than 0.9. Therefore now we will replace the missing values with the predictions of this model. We have already trained model so now we replace all null values.

In [ ]:
# Identify the rows that have a NUll values in 'Accumulated Consumption (L/day)'
missing_rows = dataset1_filtered[dataset1_filtered['Accumulated Consumption (L/day)'].isnull()]

features2 = ['Use', 'Number of meters', 'Date', 'Postcode', 'Census section']
X_ = missing_rows[features2]

# Predict the values with the model
predicted_values = knn_model.predict(X_)

dataset1_filtered.loc[missing_rows.index, 'Accumulated Consumption (L/day)'] = predicted_values

We now check that no null values nor negatives are in the dataset.

In [ ]:
# Number of null values in Accumulated Consumption (L/day)
print("Number of Nulls in the dataset:", dataset1_filtered['Accumulated Consumption (L/day)'].isnull().sum())

# Number of negatives values in Accumulated Consumption (L/day)
print("Number of Negatives in the dataset:", len(dataset1_filtered[dataset1_filtered['Accumulated Consumption (L/day)'] < 0]))

In order to keep working with our normalized values lets once again put all null values in 'Normalized Accumulated Consumption (L/day)' by diving the consumption by the number of meters.

In [ ]:
predicted_values = dataset1_filtered['Accumulated Consumption (L/day)'] / dataset1_filtered['Number of meters']

dataset1_filtered.loc[dataset1_filtered['Normalized Accumulated Consumption (L/day)'].isnull(), 'Normalized Accumulated Consumption (L/day)'] = predicted_values

In [ ]:
# Number of null values in Normalized Accumulated Consumption (L/day)
print("Number of Nulls in the dataset:", dataset1_filtered['Normalized Accumulated Consumption (L/day)'].isnull().sum())

# Number of negatives values in Normalized Accumulated Consumption (L/day)
print("Number of Negatives in the dataset:", len(dataset1_filtered[dataset1_filtered['Normalized Accumulated Consumption (L/day)'] < 0]))

Lets visualize the Consumption after having cleaned and projected the incorrect values.

In [ ]:
# PLOTS

## 5. Analysis of anomalies

##### **5.1. Classification Criteria**

Previously, we stored all anomalies for later analysis. We are now going to state a classification criteria in order to classify said anomalies into 3 categories: leak or waste, system error or correct but misclassified.
This classification is based on the average of the correct real data, I'm not using the predicted one by the models but it might not be very accurate. This should be done with specific data portraying leaks, waste and system errors.

In [ ]:
# % of anomalies with respect to the original dataset
percentage = (334898 / 4820790 ) * 100
print("% of anomalies with respect to the original dataset: ", round(percentage, 2), "%")

In [ ]:
# Count of Outliers and Negative values
outliers_count = (anomalies['Normalized Accumulated Consumption (L/day)'] > 0).sum()
negative_count = (anomalies['Normalized Accumulated Consumption (L/day)'] < 0).sum()

total_count = len(anomalies)
outliers_percentage = (outliers_count / total_count) * 100
negative_percentage = (negative_count / total_count) * 100

labels = ['Outliers', 'Negative Consumption']
sizes = [outliers_percentage, negative_percentage]
colors = ['cornflowerblue', 'lightskyblue']

# Plotting the pie chart
plt.figure(figsize=(10, 6))
plt.pie(sizes, labels=labels, colors=colors, autopct='%1.1f%%', startangle=140)
plt.title('Percentage of Outliers and Negative Consumption')
plt.axis('equal')
plt.show()

Now we compute the minimum, average and maximum values per type of Use so we can classify each values accordingly.

In [ ]:
# For each type of Use we can compute the min, average and max values
grouped_data = clean_data.groupby('Use')['Normalized Accumulated Consumption (L/day)'].agg(['min', 'mean', 'max'])

# Accessing min, average, and max values for a specific 'Use' type
use_type = 'Domèstic/Doméstico/Domestic'
domestic_min = grouped_data.loc[use_type, 'min']
domestic_average = grouped_data.loc[use_type, 'mean']
domestic_max = grouped_data.loc[use_type, 'max']

use_type = 'Comercial/Comercial/Commercial'
commercial_min = grouped_data.loc[use_type, 'min']
commercial_average = grouped_data.loc[use_type, 'mean']
commercial_max = grouped_data.loc[use_type, 'max']

use_type = 'Industrial/Industrial/Industrial'
industrial_min = grouped_data.loc[use_type, 'min']
industrial_average = grouped_data.loc[use_type, 'mean']
industrial_max = grouped_data.loc[use_type, 'max']

grouped_data

##### **5.2. Anomalies classification**

The following function gets the anomalies dataset and depending on the type of Use applies a classification. I considered consumption values over the maximum value to be of type 'Leak or Waste'. If the consumption is negative then I consider this a 'Data Collection System Error' and in case of not belonging in any of these two groups then the value would be 'Correct but Misclassified'.

In [ ]:
def anomalies_classification(dataset_anomalies):
    if dataset_anomalies['Use'] == 'Domèstic/Doméstico/Domestic':
        if dataset_anomalies['Normalized Accumulated Consumption (L/day)'] > domestic_max:
            return 'Leak or Waste'
        elif dataset_anomalies['Normalized Accumulated Consumption (L/day)'] < 0:
            return 'Data Collection System Error'
        else:
            return 'Correct but Misclassified'
        
    elif dataset_anomalies['Use'] == 'Comercial/Comercial/Commercial':
        if dataset_anomalies['Normalized Accumulated Consumption (L/day)'] > commercial_max:
            return 'Leak or Waste'
        elif dataset_anomalies['Normalized Accumulated Consumption (L/day)'] < 0:
            return 'Data Collection System Error'
        else:
            return 'Correct but Misclassified'
        
    elif dataset_anomalies['Use'] == 'Industrial/Industrial/Industrial':
        if dataset_anomalies['Normalized Accumulated Consumption (L/day)'] > industrial_max:
            return 'Leak or Waste'
        elif dataset_anomalies['Normalized Accumulated Consumption (L/day)'] < 0:
            return 'Data Collection System Error'
        else:
            return 'Correct but Misclassified'
    

anomalies['Classification'] = anomalies.apply(anomalies_classification, axis=1)
anomalies.head()

Now we can visualize the new classification in total and per type of Use.

In [ ]:
# Count occurrences of each classification
classification_counts = anomalies['Classification'].value_counts()
labels = classification_counts.index
sizes = classification_counts.values
colors = ['lightskyblue', 'blue', 'cornflowerblue']

# Plotting the pie chart
plt.figure(figsize=(8, 6))
plt.pie(sizes, labels=labels, colors=colors, autopct='%1.1f%%', startangle=140)
plt.title('Anomalies Classification')
plt.axis('equal')
plt.show()

In [ ]:
# Filter the DataFrame
domestic_anomalies = anomalies[anomalies['Use'] == 'Domèstic/Doméstico/Domestic']

# Count occurrences of each classification
classification_counts = domestic_anomalies['Classification'].value_counts()
labels = classification_counts.index
sizes = classification_counts.values
colors = ['lightskyblue', 'blue', 'cornflowerblue'] 

# Plotting the pie chart
plt.figure(figsize=(8, 6))
plt.pie(sizes, labels=labels, colors=colors, autopct='%1.1f%%', startangle=140)
plt.title('Anomalies Classification for Domestic Use')
plt.axis('equal')
plt.show()

In [ ]:
# Filter the DataFrame
domestic_anomalies = anomalies[anomalies['Use'] == 'Comercial/Comercial/Commercial']

# Count occurrences of each classification
classification_counts = domestic_anomalies['Classification'].value_counts()
labels = classification_counts.index
sizes = classification_counts.values
colors = ['lightskyblue', 'blue', 'cornflowerblue']

# Plotting the pie chart
plt.figure(figsize=(8, 6))
plt.pie(sizes, labels=labels, colors=colors, autopct='%1.1f%%', startangle=140)
plt.title('Percentage of Each Anomaly Classification for Commercial Use')
plt.axis('equal') 
plt.show()

In [ ]:
# Filter the DataFrame
domestic_anomalies = anomalies[anomalies['Use'] == 'Industrial/Industrial/Industrial']

# Count occurrences of each classification
classification_counts = domestic_anomalies['Classification'].value_counts()
labels = classification_counts.index
sizes = classification_counts.values
colors = ['lightskyblue', 'blue', 'cornflowerblue']

# Plotting the pie chart
plt.figure(figsize=(8, 6))
plt.pie(sizes, labels=labels, colors=colors, autopct='%1.1f%%', startangle=140)
plt.title('Percentage of Each Anomaly Classification for Industrial Use')
plt.axis('equal')
plt.show()

In [ ]:
# Download anomalies
anomalies.to_csv('anomalies_dataset1_v2.csv', index=False)

## 6. Export improved data

In [ ]:
dataset1_filtered.to_csv('updated_dataset1_v2.csv', index=False)